## アンサンブル学習

3種類のアンサンブル学習をスクラッチ実装していきます。そして、それぞれの効果を小さめのデータセットで確認します。


ブレンディング
バギング
スタッキング

小さなデータセットの用意
以前も利用した回帰のデータセットを用意します。


House Prices: Advanced Regression Techniques


この中のtrain.csvをダウンロードし、目的変数としてSalePrice、説明変数として、GrLivAreaとYearBuiltを使います。


train.csvを学習用（train）8割、検証用（val）2割に分割してください。


scikit-learn
単一のモデルはスクラッチ実装ではなく、scikit-learnなどのライブラリの使用を推奨します。


sklearn.linear_model.LinearRegression — scikit-learn 0.21.3 documentation


sklearn.svm.SVR — scikit-learn 0.21.3 documentation


sklearn.tree.DecisionTreeRegressor — scikit-learn 0.21.3 documentation

### 【問題1】ブレンディングのスクラッチ実装
ブレンディング をスクラッチ実装し、単一モデルより精度があがる例を 最低3つ 示してください。精度があがるとは、検証用データに対する平均二乗誤差（MSE）が小さくなることを指します。


ブレンディングとは
ブレンディングとは、N個の多様なモデルを独立して学習させ、推定結果を重み付けした上で足し合わせる方法です。最も単純には平均をとります。多様なモデルとは、以下のような条件を変化させることで作り出すものです。


手法（例：線形回帰、SVM、決定木、ニューラルネットワークなど）
ハイパーパラメータ（例：SVMのカーネルの種類、重みの初期値など）
入力データの前処理の仕方（例：標準化、対数変換、PCAなど）

重要なのはそれぞれのモデルが大きく異なることです。


回帰問題でのブレンディングは非常に単純であるため、scikit-learnには用意されていません。


《補足》


分類問題の場合は、多数決を行います。回帰問題に比べると複雑なため、scikit-learnにはVotingClassifierが用意されています。


sklearn.ensemble.VotingClassifier — scikit-learn 0.21.3 documentation

In [8]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd

/Users/apple/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [9]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
#import xgboost as xgb

lr = LinearRegression()
svm = SVR()
tree = DecisionTreeRegressor()
forest = RandomForestClassifier()
lgb = lgb.LGBMClassifier()
#xgb = xgb.XGBRegressor(colsample_bytree = 0.5, learning_rate =0.1, max_depth = 2, n_estimators = 50,subsample =  1)

/Users/apple/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/apple/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/lightgbm/__init__.py:48: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [10]:
df = pd.read_csv("train.csv")

df_selected=df.loc[:, ["GrLivArea", "YearBuilt", "SalePrice"]]
df_selected.head()



,GrLivArea,YearBuilt,SalePrice
0,1710,2003,208500
1,1262,1976,181500
2,1786,2001,223500
3,1717,1915,140000
4,2198,2000,250000


In [11]:
X=df_selected.iloc[:, :-1]
X.head()

,GrLivArea,YearBuilt
0,1710,2003
1,1262,1976
2,1786,2001
3,1717,1915
4,2198,2000


In [12]:
y=df_selected.loc[:, "SalePrice"]
y.head()

0    208500
1    181500
2    223500
3    140000
4    250000
Name: SalePrice, dtype: int64

In [13]:
#! pip install xgboost scikit-learn matplotlib

In [14]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(
              X , y , test_size = 0.2,random_state=1)

In [15]:
def MSE(y_test , pred):
    return mean_squared_error(y_test , pred)

In [16]:
lr = LinearRegression()
svm = SVR()
tree = DecisionTreeRegressor()


SVMのSMEの値が大きい

In [17]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(
              X , y , test_size = 0.2, random_state=1)

lr.fit(X_train , y_train)
svm.fit(X_train , y_train)
tree.fit(X_train , y_train)
pred_lr = lr.predict(X_test)
pred_svm = svm.predict(X_test)
pred_tree = tree.predict(X_test)

mse_lr = MSE(y_test , pred_lr)
mse_svm = MSE(y_test , pred_svm)
mse_tree = MSE(y_test , pred_tree)
pred_mean = (pred_lr+pred_svm+pred_tree )/3
mse_mean = MSE(y_test, pred_mean)


In [18]:
print("線形回帰のSME : {:.2f}".format(mse_lr))
print("スコア{}".format(lr.score(X_test , y_test)))
print("SVMのSME : {:.2f}".format(mse_svm))
print("決定木のSME : {:.1e}".format(mse_tree))
print("平均SME : {:.2f}".format(mse_mean))

線形回帰のSME : 2000178029.14
スコア0.7195465634622643
SVMのSME : 7249158070.01
決定木のSME : 2.4e+09
平均SME : 2371526284.84


In [19]:
pred_weight = (pred_lr*3 + pred_svm+ pred_tree*2)/6
mse_weight = MSE(y_test, pred_weight)

print("重みSME : {:.2f}".format(mse_weight))

重みSME : 1904971094.17


平均では線形回帰より良いSMEが出なかったが、重みをつけることでSMEの良い値が出た。

In [20]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(
              X , y , test_size = 0.2, random_state=1)

lr.fit(X_train , y_train)
lgb.fit(X_train , y_train)
tree.fit(X_train , y_train)
pred_lr = lr.predict(X_test)
pred_lgb = lgb.predict(X_test)
pred_tree = tree.predict(X_test)

mse_lr = MSE(y_test , pred_lr)
mse_lgb = MSE(y_test , pred_lgb)
mse_tree = MSE(y_test , pred_tree)
pred_mean = (pred_lr+pred_lgb+pred_tree )/3
mse_mean = MSE(y_test, pred_mean)

In [21]:
print("線形回帰のSME : {:.2f}".format(mse_lr))
print("lgbのSME : {:.2f}".format(mse_lgb))
print("決定木のSME : {:.2f}".format(mse_tree))
print("平均SME : {:.2f}".format(mse_mean))


線形回帰のSME : 2000178029.14
lgbのSME : 3121938883.20
決定木のSME : 2632857997.19
平均SME : 1885938167.05


平均で線形回帰より良いSMEが出た。

In [22]:
pred_weight = (pred_lr*3 + pred_lgb + pred_tree*2)/6
mse_weight = MSE(y_test, pred_weight)

print("重みSME : {:.2f}".format(mse_weight))

重みSME : 1732554061.58


重みをつけることでSMEの良い値が出た。

### 【問題2】バギングのスクラッチ実装
バギング をスクラッチ実装し、単一モデルより精度があがる例を 最低1つ 示してください。


バギングとは
バギングは入力データの選び方を多様化する方法です。学習データから重複を許した上でランダムに抜き出すことで、N種類のサブセット（ ブートストラップサンプル ）を作り出します。それらによってモデルをN個学習し、推定結果の平均をとります。ブレンディングと異なり、それぞれの重み付けを変えることはありません。


sklearn.model_selection.train_test_split — scikit-learn 0.21.3 documentation


scikit-learnのtrain_test_splitを、shuffleパラメータをTrueにして使うことで、ランダムにデータを分割することができます。これによりブートストラップサンプルが手に入ります。


推定結果の平均をとる部分はブースティングと同様の実装になります。

In [23]:
def bagging(X,y,n = 10):
    
    X_train , X_test ,y_train ,  y_test = train_test_split(X , y , test_size = 0.2, random_state=1)
    pred_box=np.zeros((n,X_test.shape[0]))
    print("pred_boxのshape: {}".format(pred_box.shape))
    for i in range(n):
        
        '''
        train_test_splitは外へ
        model = Model()は中へ
        predはno.zerosで作った入れ物pred_blockに差し込んでいく
        pred_blockについてforの外で平均取る
        '''
        X_index = np.random.choice(X_train.index , round(X_train.shape[0]*0.8) , replace=True)#ランダムでXのインデックスを抜き出す。配列の要素をランダムに取り出す関数
        #print(X_index)
        #np.random.choice(取りうる値, サイズ)
        tree = DecisionTreeRegressor()
        X_train_n = X_train.loc[X_index]
        y_train_n=y_train.loc[X_index]
        tree.fit(X_train_n , y_train_n)
        pred=tree.predict(X_test)#抜き出したインデックスに対応するX_trainを作成
        pred_box[i, :]=pred#predをpred_boxに追加
    #print(pred_box)
#     print(len(pred_list))
    pred_mean=(np.mean(pred_box,axis=0))
    print("pred_meanのshape: {}".format(pred_mean.shape))
    mse = mean_squared_error(y_test , pred_mean)#MSE
#     sme_list.append(mse)#MSEをリストに追加
    #print("スコア: {}".format(tree.score(X_test , y_test)))
    print("MSE: {:.1e}".format(mse))

In [24]:
bagging(X, y)
#SMEが上がった
#スコアはマイナスでいいのか
#yは大きい数字→対数変換、正規分布に近づく、logは大きい値は緩やかに変化
#正規分布：線形回帰、平均二乗和誤差は正規分布を元にしている
#正規分布：対数変換、標準化→まずは可視化

pred_boxのshape: (10, 292)
pred_meanのshape: (292,)
MSE: 1.8e+09


In [25]:
def bagging(X,y,n = 10):
    
    X_train , X_test ,y_train ,  y_test = train_test_split(X , y , test_size = 0.2, random_state=1)
    pred_box=np.zeros((n,X_test.shape[0]))
    print("pred_boxのshape: {}".format(pred_box.shape))
    for i in range(n):
        
        '''
        train_test_splitは外へ
        model = Model()は中へ
        predはno.zerosで作った入れ物pred_blockに差し込んでいく
        pred_blockについてforの外で平均取る
        '''
        X_index = np.random.choice(X_train.index , X_train.shape[0] , replace=True)#ランダムでXのインデックスを抜き出す。配列の要素をランダムに取り出す関数
        #np.random.choice(取りうる値, サイズ)
        tree = DecisionTreeRegressor()
        X_train_n = X_train.loc[X_index]
        tree.fit(X_train_n , y_train)
        pred=tree.predict(X_test)#抜き出したインデックスに対応するX_trainを作成
        pred_box[i, :]=pred#predをpred_boxに追加
    #print(pred_box)
#     print(len(pred_list))
    pred_mean=(np.mean(pred_box,axis=0))
    print("pred_meanのshape: {}".format(pred_mean.shape))
    sme = mean_squared_error(y_test , pred_mean)#SME
#     sme_list.append(sme)#SMEをリストに追加
    print("スコア: {}".format(tree.score(X_test , y_test)))
    print("SME: {:.1e}".format(sme))

### 【問題3】スタッキングのスクラッチ実装
スタッキング をスクラッチ実装し、単一モデルより精度があがる例を 最低1つ 示してください。


スタッキングとは
スタッキングの手順は以下の通りです。最低限ステージ0とステージ1があればスタッキングは成立するため、それを実装してください。まずは 
K
0
=
3
,
M
0
=
2
 程度にします。


《学習時》


（ステージ 
0
 ）


学習データを 
K
0
 個に分割する。
分割した内の 
(
K
0
−
1
)
 個をまとめて学習用データ、残り 
1
 個を推定用データとする組み合わせが 
K
0
 個作れる。
あるモデルのインスタンスを 
K
0
 個用意し、異なる学習用データを使い学習する。
それぞれの学習済みモデルに対して、使っていない残り 
1
 個の推定用データを入力し、推定値を得る。（これをブレンドデータと呼ぶ）
さらに、異なるモデルのインスタンスも 
K
0
 個用意し、同様のことを行う。モデルが 
M
0
 個あれば、 
M
0
 個のブレンドデータが得られる。

（ステージ 
n
 ）


ステージ 
n
−
1
 のブレンドデータを
M
n
−
1
 次元の特徴量を持つ学習用データと考え、 
K
n
 個に分割する。以下同様である。

（ステージ 
N
 ）＊最後のステージ


ステージ 
N
−
1
 の 
M
N
−
1
 個のブレンドデータを
M
N
−
1
 次元の特徴量の入力として、1種類のモデルの学習を行う。これが最終的な推定を行うモデルとなる。

《推定時》


（ステージ 
0
 ）


テストデータを 
K
0
×
M
0
 個の学習済みモデルに入力し、
K
0
×
M
0
 個の推定値を得る。これを 
K
0
 の軸で平均値を求め 
M
0
 次元の特徴量を持つデータを得る。（ブレンドテストと呼ぶ）

（ステージ 
n
 ）


ステージ 
n
−
1
 で得たブレンドテストを 
K
n
×
M
n
 個の学習済みモデルに入力し、
K
n
×
M
n
 個の推定値を得る。これを 
K
n
 の軸で平均値を求め 
M
0
 次元の特徴量を持つデータを得る。（ブレンドテストと呼ぶ）

（ステージ 
N
 ）＊最後のステージ


ステージ 
N
−
1
 で得たブレンドテストを学習済みモデルに入力し、推定値を得る。

In [63]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
X_train , X_test ,y_train ,  y_test = train_test_split(X , y , test_size = 0.2, random_state=1)
kf = KFold(n_splits = 3 , random_state = 0)
for (tr_idx , va_idx) in (kf.split(X_train)):
    print("tr_idx{}".format(tr_idx))
    print("va_idx{}".format(va_idx))

tr_idx[ 390  391  392  393  394  395  396  397  398  399  400  401  402  403
  404  405  406  407  408  409  410  411  412  413  414  415  416  417
  418  419  420  421  422  423  424  425  426  427  428  429  430  431
  432  433  434  435  436  437  438  439  440  441  442  443  444  445
  446  447  448  449  450  451  452  453  454  455  456  457  458  459
  460  461  462  463  464  465  466  467  468  469  470  471  472  473
  474  475  476  477  478  479  480  481  482  483  484  485  486  487
  488  489  490  491  492  493  494  495  496  497  498  499  500  501
  502  503  504  505  506  507  508  509  510  511  512  513  514  515
  516  517  518  519  520  521  522  523  524  525  526  527  528  529
  530  531  532  533  534  535  536  537  538  539  540  541  542  543
  544  545  546  547  548  549  550  551  552  553  554  555  556  557
  558  559  560  561  562  563  564  565  566  567  568  569  570  571
  572  573  574  575  576  577  578  579  580  581  582  583  584  585


In [108]:
#     Stacking(models,X_train , X_test , y_train)
kf = KFold(n_splits = 4 , random_state = 0)
models = [lr, tree]  
pred_data = []#予測値を入れる箱
preds_test_data = []#テストデータの予測値を入れる箱
fit_data = []

#クロスバリデーション、学習、予測を行う。予測値を保存
for model in models:#indexと要素をmとmodelに入れる
    for (tr_idx , va_idx) in (kf.split(X_train)):#X_trainをkfoldして、訓練データとバリデーションデータのindexを取得
        tr_X , va_X = X_train.iloc[tr_idx] , X_train.iloc[va_idx]
        tr_y , va_y = y_train.iloc[tr_idx] , y_train.iloc[va_idx]
        model.fit(tr_X , tr_y) #trainデータで学習
        fit_data.append(model)
        pred = model.predict(va_X) #バリデーションデータで予測
        #print(pred)
        pred_data.append(pred) #予測値を追加
        pred_test = model.predict(X_test)#テストデータの予測
        preds_test_data.append(pred_test)#テストデータの予測値を追加
#     print("pred_data\n{}".format(pred_data))
#     print("preds_test_data\n{}".format(preds_test_data))
print(np.array(pred_data).shape)
print(np.array(preds_test_data).shape)
print(len(fit_data))

# Stacking(models,X_train , X_test , y_train)  
#pred_data8個のarray：→どう形を変えるか
#preds_test_data８個のarray:モデルごとに平均を取る


(8, 292)
(8, 292)
8


In [74]:
df_pred_data=np.array(pred_data)
df_preds_test_data=np.array(preds_test_data)

In [76]:
df_pred_data

array([[180703.50430549,  91593.32928865, 193580.41882989, ...,
        225671.5828609 , 291607.66991311, 249528.11428175],
       [304867.04803585, 174056.36507812, 212478.12071005, ...,
        197325.49285722, 147163.58595146, 189515.3300121 ],
       [174474.61715875, 285676.41822792, 151650.69890998, ...,
        139655.3198776 , 225976.38336613, 247916.01069959],
       ...,
       [250000.        , 165500.        , 226000.        , ...,
        165000.        , 153000.        , 192000.        ],
       [185750.        , 335000.        , 148000.        , ...,
        128950.        , 185000.        , 180000.        ],
       [250000.        , 125500.        , 156000.        , ...,
        127500.        ,  85400.        ,  91300.        ]])

In [91]:
#モデル１のブレンドデータ
df_pred_data_list=np.array([])
for i in range(4):
    print(df_pred_data[i].shape)
    df_pred_data_list=np.append(df_pred_data_list,df_pred_data[i])
print(df_pred_data_list.shape)

(292,)
(292,)
(292,)
(292,)
(1168,)


In [90]:
#モデル２のブレンドデータ
df_pred_data_list_2=np.array([])
for i in range(4,8):
    print(df_pred_data[i].shape)
    df_pred_data_list_2=np.append(df_pred_data_list_2,df_pred_data[i])
print(df_pred_data_list_2.shape)

(292,)
(292,)
(292,)
(292,)
(1168,)


In [101]:
#ブレンドデータの結合
df_blend_data = np.concatenate([df_pred_data_list.reshape(-1,1),df_pred_data_list_2.reshape(-1,1)],axis=1)

df_blend_data.shape

(1168, 2)

In [136]:
df_pred_data_list.reshape(-1,1).shape

(1168, 1)

In [149]:
df_pred_data_list_2.reshape(-1,1).shape

(1168, 1)

In [105]:
#ブレンドデータの学習
lr.fit(df_blend_data ,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [156]:
#testdata
preds_test_data_2 = np.empty((292,0))#テストデータの予測値を入れる箱

for i in range(len(fit_data)):    
    pred_test_2 = fit_data[i].predict(X_test)#テストデータの予測
    preds_test_data_2=np.concatenate([preds_test_data_2,pred_test_2.reshape(-1,1)],axis=1)#テストデータの予測値を追加
    #df_pred_data_list_2=np.append(df_pred_data_list_2,df_pred_data[i])
print(preds_test_data_2.shape)
pred_test_2.shape

(292, 8)


(292,)

In [152]:
preds_test_data_2.reshape(-1,1).shape

(292, 1)

In [162]:

ptd_1=preds_test_data_2[:,:4]
preds_test_data_2[:,:4].shape


(292, 4)

In [174]:
ptd_1_mean=np.mean(ptd_1, axis=1)
ptd_1_mean.shape

(292,)

In [169]:
ptd_2=preds_test_data_2[:,4:]
preds_test_data_2[:,4:].shape


(292, 4)

In [175]:
ptd_2_mean=np.mean(ptd_2, axis=1)
ptd_2_mean.shape

(292,)

In [180]:
ptd_con=np.concatenate([ptd_1_mean.reshape(-1,1),ptd_2_mean.reshape(-1,1)],axis=1)
ptd_con.shape

(292, 2)

In [182]:
ptd_predict=lr.predict(ptd_con)
ptd_predict.shape

(292,)